In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader,random_split
from torch.optim.lr_scheduler import StepLR,ReduceLROnPlateau
import torchmetrics.functional as metrics
import os
import shutil
from torchvision import transforms
from PIL import Image

/Users/sml/anaconda3/envs/Torch_PY38/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/sml/anaconda3/envs/Torch_PY38/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/sml/anaconda3/envs/Torch_PY38/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <6A598D74-186E-3808-8921-63BA99511723> /Users/sml/anaconda3/envs/Torch_PY38/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# 학습 검증용 데이터
folder_path = '../train/train' # 
target_data = []
img_data = []
for encoding_label,label in enumerate(os.listdir(folder_path)):
    label_path = os.path.join(folder_path, label)
    if os.path.isdir(label_path):  # 디렉토리인 경우에만 진입
        for img in os.listdir(folder_path+'/'+label):
            image_path = os.path.join(folder_path,label,img)
            if os.path.isfile(image_path):  # 파일인 경우에만 진입
                with open(image_path, 'rb') as file:
                    image = Image.open(file)
                    width, height = image.size
                    if width == 48 and height == 48:
                        image_array = np.array(image)
                        target_data.append(encoding_label)
                        img_data.append(image_array)

In [3]:
# 테스트용 데이터
folder_path = '../test/test'
target_test = []
img_test = []
for encoding_label,label in enumerate(os.listdir(folder_path)):
    label_path = os.path.join(folder_path, label)
    if os.path.isdir(label_path):  # 디렉토리인 경우에만 진입
        for img in os.listdir(folder_path+'/'+label):
            image_path = os.path.join(folder_path,label,img)
            if os.path.isfile(image_path):  # 파일인 경우에만 진입
                with open(image_path, 'rb') as file:
                    image = Image.open(file)
                    width, height = image.size
                    if width == 48 and height == 48:
                        image_array = np.array(image)
                        target_test.append(encoding_label)
                        img_test.append(image_array)

In [4]:
# 이미지 데이터 정규화
x_data = np.array(img_data)/255.
x_data = x_data.reshape((-1,48*48))
print(x_data.shape)

(28709, 2304)


In [5]:
x_data_test = np.array(img_test)/255.
x_data_test = x_data_test.reshape((-1,48*48))
print(x_data_test.shape)

(7178, 2304)


In [6]:
target_data = pd.Series(target_data).replace({0:3, 5:4, 2:5, 3:2, 6:0, 4:6, 7:1})
target_data.value_counts()

3    7215
4    4965
5    4830
2    4097
0    3995
6    3171
1     436
Name: count, dtype: int64

In [7]:
target_test = pd.Series(target_test).replace({0:3, 5:4, 2:5, 3:2, 6:0, 4:6, 7:1})
target_test.value_counts()

3    1774
1    1247
6    1233
5    1024
4     958
2     831
0     111
Name: count, dtype: int64

In [8]:
# 데이터 클래스 생성
class DLdataset(Dataset):
    
    def __init__(self,x_data,y_data):
        super().__init__()
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)
        
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self,idx):
        return self.feature[idx], self.target[idx]

In [9]:
# 데이터셋 생성
dataset = DLdataset(x_data,target_data)
dataset_test = DLdataset(x_data_test, target_test)

In [10]:
# 학습용, 검증용 데이터 준비
seed = torch.Generator().manual_seed(42)
trainDS, validDS = random_split(dataset, [0.8,0.2], generator=seed)


In [11]:
# 배치사이즈 32
BATCH = 32
trainDL = DataLoader(trainDS, batch_size=BATCH)
validDL = DataLoader(validDS, batch_size=BATCH)
testDL = DataLoader(dataset_test, batch_size=BATCH)


In [12]:
# 모델 클래스 정의
class Model(nn.Module):
    
    def __init__(self, IN, OUT):
        super().__init__()
        self.input = nn.Linear(IN, 128) 
        self.af = nn.ReLU()
        self.hidden = nn.Linear(128, 32)
        self.output = nn.Linear(32, OUT)
        
    def forward(self, x):
        y = self.input(x)
        y = self.af(y)
        y = self.hidden(y)
        y = self.af(y)
        y = self.output(y)
        
        return y

In [13]:
# 학습 준비

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

EPOCHS = 5

IN = dataset.feature.shape[1]
OUT = pd.Series(target_data).nunique()

# 모델 생성
model  = Model(IN,OUT)

# 손실함수
LF = nn.CrossEntropyLoss().to(DEVICE)

# 옵티마이저
OPTIMIZER = torch.optim.Adam(model.parameters())

# 스케줄러
SCHEDULER = ReduceLROnPlateau(OPTIMIZER, mode = 'min', patience = 3)

In [14]:
def training(dataLoader):
    
    model.train()
    train_report=[[], []]
    for (feature, target) in dataLoader:

        feature, target = feature.to(DEVICE), target.to(DEVICE)
        
        # 학습
        pre_target = model(feature)
        
        # 손실계산
        loss = LF(pre_target, target)
        train_report[0].append(loss)
  
        # 성능 평가
        acc = metrics.accuracy(pre_target.argmax(dim=1), target, task = 'multiclass',num_classes=OUT)
        train_report[1].append(acc)
        
        # W,b업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()

    loss_score = sum(train_report[0])/len(train_report[0])
    acc_score = sum(train_report[1])/len(train_report[1])
    print(f'[Train loss] ==> {loss_score}    [Train Accuracy] ==> {acc_score}')
    return loss_score, acc_score

In [15]:
def testing(dataLoader):
    
    model.eval()
    
    with torch.no_grad():
        test_report=[[], []]
        for (feature, target)  in dataLoader:
            # 배치크기만큼의 학습 데이터 준비
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            
            # 학습
            pre_target = model(feature)
            
            # 손실계산
            loss = LF(pre_target, target)
            test_report[0].append(loss)
      
            # 성능 평가
            acc = metrics.accuracy(pre_target.argmax(dim=1), target, task = 'multiclass',num_classes=OUT)
            test_report[1].append(acc)
    
    loss_score = sum(test_report[0])/len(test_report[0])
    acc_score = sum(test_report[1])/len(test_report[1])

    # print(f'[Test loss] ==> {loss_score}    [Test Accuracy] ==> {acc_score}')
    return loss_score, acc_score

In [68]:
def predicting(dataset,n):

    model = Model(IN, OUT)  # 같은 모델 구조의 객체 생성
    model_path = "my_trained_model.pth"
    model.load_state_dict(torch.load(model_path))
    model.eval()

    correct = 0
    total = 0

    for feature, target in dataset:
        # img, ytrue = dataLoader[idx][0], dataLoader[idx][1]

        with torch.no_grad():
            ypre = model(img.unsqueeze(0))
            ypre = torch.argmax(ypre, dim=1).item()

            if img.shape[0] == 2304:
                img = img.reshape(48, 48)

            if ypre == ytrue:
                correct += 1

            total += 1
            # if idx in[1, 1000, 2000, 3000, 4000, 5000, 6000, 7000] :
            plt.imshow(img.numpy().reshape(48, 48), cmap='gray_r')
            plt.title(f'[{idx+1}] True {ytrue} / Predict {ypre}')
            plt.xticks([])
            plt.yticks([])
            plt.show()


    accuracy = correct / total
    print(f'Accuracy: {accuracy:.2f}, Correct : {correct}, Total : {total}')
    return accuracy

In [71]:
len(dataset_test)

7178

In [ ]:
# def predicting(dataLoader,n,Model,filename):
# # 0:angry / 1:disgust / 2:fear / 3:happy / 4:neutral / 5:sad / 6:surprise
#     label = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
#     model_path = 'model/'+filename
#     Model.load_state_dict(torch.load(model_path))
#     Model.eval()

#     correct = 0
#     total = 0

#     for idx in range(len(dataset)):
#         img, ytrue = dataset[idx][0], dataset[idx][1]

#         with torch.no_grad():
#             ypre = Model(img.unsqueeze(0))
#             ypre = torch.argmax(ypre, dim=1).item()

#             if img.shape[0] == 2304:
#                 img = img.reshape(48, 48)

#             if ypre == ytrue:
#                 correct += 1
            
#             total += 1
#             if idx < n :
#                 plt.imshow(dataset[idx][0].numpy().reshape(48, 48), cmap='gray_r')
#                 plt.title(f'[{idx+1}] True {label[ytrue]} / Predict {label[ypre]}')
#                 plt.xticks([])
#                 plt.yticks([])
#                 plt.show()


#     accuracy = correct / total
#     print(f'Accuracy: {accuracy:.2f}, Correct : {correct}, Total : {total}')
#     return accuracy

In [49]:
# min_loss = 100.0  # 초기 최소 손실 설정
# cnt = 0
# for eps in range(EPOCHS):
#     print(f'[{eps+1}/{EPOCHS}]')
#     # 학습
#     train_loss, train_acc = training(trainDL)

#     # 검증
#     val_loss, val_acc = testing(validDL)
    
#     # 최소 손실 업데이트
#     if val_loss < min_loss:
#         min_loss = val_loss
#         cnt = 0
#         torch.save(model.state_dict(), "my_trained_model.pth")

#     else:
#         cnt+=1

#     # 조기 종료 기능 => 조건 : val_loss가 지정된 횟수 이상 개선이 안되면 학습 종료
#     if SCHEDULER.num_bad_epochs >= SCHEDULER.patience or cnt >= 5:
#         print(f"Early stopping at epoch {eps}")
#         break

In [50]:
training(trainDL)

AttributeError: 'SVC' object has no attribute 'train'

In [51]:
testing(validDL)

AttributeError: 'SVC' object has no attribute 'eval'

In [61]:
testDL

TypeError: 'DataLoader' object is not subscriptable

In [67]:
predicting(testDS)

TypeError: 'DataLoader' object is not subscriptable

In [53]:

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

EPOCHS = 100

IN = dataset.feature.shape[1]
OUT = pd.Series(target_data).nunique()

# 모델 생성
model  = Model(IN,OUT)

# 손실함수
LF = nn.CrossEntropyLoss().to(DEVICE)

OPTIMIZERLIST = [
    ("GD", optim.SGD),      # Stochastic Gradient Descent
    # ("Momentum", optim.SGD),       
    # ("NAG", optim.SGD, {'momentum': 0.9, 'nesterov': True}), 
    ("Adagrad", optim.Adagrad),
    ("RMSProp", optim.RMSprop),
    # ("Nadam", optim.Nadam),  
    ("AdaDelta", optim.Adadelta),
    ("Adam", optim.Adam)
]

GDLIST = [[],[],[],[]] 
ADAGRADLIST = [[],[],[],[]]
RMSPROPLIST = [[],[],[],[]]
ADADELTALIST = [[],[],[],[]]
ADAMLIST = [[],[],[],[]]


def training(dataLoader):
    
    model.train()
    train_report=[[], []]
    for (feature, target) in dataLoader:

        feature, target = feature.to(DEVICE), target.to(DEVICE)
        
        # 학습
        pre_target = model(feature)
        
        # 손실계산
        loss = LF(pre_target, target)
        train_report[0].append(loss)

        # 성능 평가
        acc = metrics.accuracy(pre_target.argmax(dim=1), target, task = 'multiclass',num_classes=OUT)
        train_report[1].append(acc)
        
        # W,b업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()

    loss_score = sum(train_report[0])/len(train_report[0])
    acc_score = sum(train_report[1])/len(train_report[1])
    # print(f'[Train loss] ==> {loss_score}    [Train Accuracy] ==> {acc_score}')
    return loss_score, acc_score
try:
    for name, OPTI in OPTIMIZERLIST:
        print(f"\n--- Training with {name} optimizer ---")



        # 옵티마이저
        OPTIMIZER = OPTI(model.parameters())

        # 스케줄러
        SCHEDULER = ReduceLROnPlateau(OPTIMIZER, mode = 'min', patience = 3)

        min_loss = 100.0  # 초기 최소 손실 설정
        cnt = 0
        for eps in range(EPOCHS):
            # print(f'[{eps+1}/{EPOCHS}]')
            # 학습
            train_loss, train_acc = training(trainDL)

            # 검증
            val_loss, val_acc = testing(validDL)
            
            # 최소 손실 업데이트
            if val_loss < min_loss:
                min_loss = val_loss
                cnt = 0
                torch.save(model.state_dict(), name+' '+"my_trained_model.pth")

            else:
                cnt+=1


            if name == 'GD':
                GDLIST[0].append(train_loss)
                GDLIST[1].append(train_acc)
                GDLIST[2].append(val_loss)
                GDLIST[3].append(val_acc)
            elif name == 'Adagrad':
                ADAGRADLIST[0].append(train_loss)
                ADAGRADLIST[1].append(train_acc)
                ADAGRADLIST[2].append(val_loss)
                ADAGRADLIST[3].append(val_acc)
            elif name == 'RMSProp':
                RMSPROPLIST[0].append(train_loss)
                RMSPROPLIST[1].append(train_acc)
                RMSPROPLIST[2].append(val_loss)
                RMSPROPLIST[3].append(val_acc)
            elif name == 'AdaDelta':
                ADADELTALIST[0].append(train_loss)
                ADADELTALIST[1].append(train_acc)
                ADADELTALIST[2].append(val_loss)
                ADADELTALIST[3].append(val_acc)
            elif name == 'Adam':
                ADAMLIST[0].append(train_loss)
                ADAMLIST[1].append(train_acc)
                ADAMLIST[2].append(val_loss)
                ADAMLIST[3].append(val_acc)


            # 조기 종료 기능 => 조건 : val_loss가 지정된 횟수 이상 개선이 안되면 학습 종료
            if SCHEDULER.num_bad_epochs >= SCHEDULER.patience or cnt >= 5:
                print(f'[{eps+1}/{EPOCHS}] {name} count : {cnt}')
                print(f'Train_loss : {train_loss}, Train_accuracy : {train_acc}')
                print(f'Val_loss : {val_loss}, Val_accuracy : {val_acc}')
                print(f"Early stopping at epoch {eps}")
                break
        print(f'[{eps+1}/{EPOCHS}] {name} count : {cnt}')
        print(f'Train_loss : {train_loss}, Train_accuracy : {train_acc}')
        print(f'Val_loss : {val_loss}, Val_accuracy : {val_acc}')
except Exception as e:
    print(e)




--- Training with GD optimizer ---


KeyboardInterrupt: 

In [42]:
for g in GDLIST:
    print(g, end='\n\n')

[tensor(1.8687, grad_fn=<DivBackward0>), tensor(1.8234, grad_fn=<DivBackward0>), tensor(1.8129, grad_fn=<DivBackward0>), tensor(1.8081, grad_fn=<DivBackward0>), tensor(1.8043, grad_fn=<DivBackward0>), tensor(1.8005, grad_fn=<DivBackward0>), tensor(1.7967, grad_fn=<DivBackward0>), tensor(1.7928, grad_fn=<DivBackward0>), tensor(1.7887, grad_fn=<DivBackward0>), tensor(1.7845, grad_fn=<DivBackward0>), tensor(1.7801, grad_fn=<DivBackward0>), tensor(1.7755, grad_fn=<DivBackward0>), tensor(1.7706, grad_fn=<DivBackward0>), tensor(1.7652, grad_fn=<DivBackward0>), tensor(1.7597, grad_fn=<DivBackward0>), tensor(1.7539, grad_fn=<DivBackward0>), tensor(1.7480, grad_fn=<DivBackward0>), tensor(1.7418, grad_fn=<DivBackward0>), tensor(1.7356, grad_fn=<DivBackward0>), tensor(1.7294, grad_fn=<DivBackward0>), tensor(1.7234, grad_fn=<DivBackward0>), tensor(1.7176, grad_fn=<DivBackward0>), tensor(1.7121, grad_fn=<DivBackward0>), tensor(1.7069, grad_fn=<DivBackward0>), tensor(1.7021, grad_fn=<DivBackward0>),

In [43]:
for a in ADAMLIST:
    print(a)

[tensor(1.8096, grad_fn=<DivBackward0>), tensor(1.8094, grad_fn=<DivBackward0>), tensor(1.8093, grad_fn=<DivBackward0>), tensor(1.8093, grad_fn=<DivBackward0>), tensor(1.8093, grad_fn=<DivBackward0>), tensor(1.8093, grad_fn=<DivBackward0>), tensor(1.8093, grad_fn=<DivBackward0>), tensor(1.8093, grad_fn=<DivBackward0>), tensor(1.8093, grad_fn=<DivBackward0>)]
[tensor(0.2512), tensor(0.2512), tensor(0.2512), tensor(0.2512), tensor(0.2512), tensor(0.2512), tensor(0.2512), tensor(0.2512), tensor(0.2512)]
[tensor(1.8109), tensor(1.8107), tensor(1.8106), tensor(1.8106), tensor(1.8106), tensor(1.8107), tensor(1.8107), tensor(1.8107), tensor(1.8107)]
[tensor(0.2528), tensor(0.2528), tensor(0.2528), tensor(0.2528), tensor(0.2528), tensor(0.2528), tensor(0.2528), tensor(0.2528), tensor(0.2528)]


In [39]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaler.fit(x_data)
# scaled_X_train = scaler.transform(x_data)
# scaled_x_test = scaler.transform(x_data_test)

In [41]:
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# model = SVC(kernel='linear')
# model.fit(scaled_X_train, target_data)
# train_score = model.score(scaled_X_train, target_data)
# test_score = model.score(scaled_x_data, target_test)

# print(f'Train score : {train_score}, Test score : {test_score}')

# y_pred = model.predict(scaled_x_test)
# accuracy = accuracy_score(target_test, y_pred)

# print(f'{accuracy}')